# Travail réalisé par : KHELOUI Gaya
## 3525803

### Import des bibliothèques necessaires

In [91]:
import torch
import torch.utils.data as dt
import numpy as np
import random
import pylab as plb
from scipy.special import logit 
import  matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.autograd import Function
import matplotlib.pyplot as plt
from torch.autograd import Variable

## Chargement des données

In [92]:
## une fois le dataset telecharge, mettre download=False !
## Pour le test, train = False
## transform permet de faire un preprocessing des donnees (ici ?)
batch_size=64
nb_digits=10
test_loader = dt.DataLoader(datasets.MNIST('../data', train=False, download=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])), batch_size=batch_size, shuffle=True) 
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])), batch_size=batch_size, shuffle=True) 

## Fonctions utiles

In [93]:
def accuracy(ypred,yreal) :
    ypred_ = transform(ypred)
    yreal_ = transform(yreal)
    return sum((torch.eq(ypred_,yreal_)))

def transform(ypredict) :
    y_return = torch.FloatTensor(ypredict.size()[0])
    y_return.zero_()
    for i in range(ypredict.size()[0]) :
        valeur, indice_max = torch.max(ypredict[i],0)
        y_return[i] = indice_max.data[0]
    return y_return

def calcul_accuracy(loader,modules,batch_size,nb_digits) :
    resultat = 0
    y_onehot = torch.FloatTensor(batch_size,nb_digits)
    for i,(data,target) in enumerate(loader):
        if( i < len(loader) - 1) :
            y_onehot = torch.FloatTensor(batch_size,nb_digits)
            y_onehot.zero_()
            y_onehot.scatter_(1, target.view(-1,1), 1)
            y_onehot = Variable(y_onehot,requires_grad=False) 

            data = Variable(data.view(-1 , 28*28))
            
            ypredict = modules(data) 
            acc = accuracy(ypredict,y_onehot)
            resultat += acc
    return resultat / (len(loader) * batch_size * 1.0)

## Classe du modele liniaire

In [94]:
class Linear_Model_tanh(torch.nn.Module):
    def __init__(self , X ,Y) :
        torch.nn.Module.__init__(self)
        self.modele = torch.nn.Linear(X,Y)
        
        
    def forward(self , x):
        pred = torch.tanh(self.modele(x))
        return pred

## Classe du modèle de transformation

In [115]:
class Transform_Model_sigm(torch.nn.Module):
    def __init__(self , X ,Y) :
        torch.nn.Module.__init__(self)
        self.modele = torch.nn.Linear(X,Y)
        
        
    def forward(self , x):
        return (torch.sigmoid(self.modele(x)))

## Classe représentant une couche HighWay

In [96]:
class Couche_HighWay(torch.nn.Module):
    def __init__(self , X ,Y) :
        torch.nn.Module.__init__(self)
        self.trans_mod = Transform_Model_sigm(X,Y)
        self.lin_mod = Linear_Model_tanh(X,Y)
        
    def forward(self,x) :
        pred = self.lin_mod(x)
        mask1 = self.trans_mod(x)
        one = Variable(torch.FloatTensor(np.ones((mask1.size()[0],mask1.size()[1]))))
        pred_grad = pred.mul(mask1)
        mask2 = one.sub(mask1)
        sign_gard = x.mul(mask2)
        pred = pred_grad.add(sign_gard)
        return pred

## Classe Représentant La partie HighWay du réseau

In [119]:
class HightWay_network(torch.nn.Module):
    def __init__(self , X ,Y,nb_couches) :
        torch.nn.Module.__init__(self)
        self.liste_modules = torch.nn.Sequential()
        self.nb_couches = nb_couches
        for i in range (nb_couches) :
            self.liste_modules.add_module(''+str(i),Couche_HighWay(X,Y))
                
                
    def forward(self , x):
        data_pred = self.liste_modules(x)
        return data_pred    

# Expérimentation 

## Profondeur = 5

### HighWay NetWork

In [130]:
taille_image  = 28*28
nb_digits = 10
dim_highWay_network = 200
modele = torch.nn.Sequential(Linear_Model_tanh(taille_image,dim_highWay_network),HightWay_network(dim_highWay_network,dim_highWay_network,5),
                            Linear_Model_tanh(dim_highWay_network ,nb_digits))  
l = torch.nn.MSELoss(size_average=False)
step = 0.005
tab_err = []
tab_acc = []
tab_acc_test = []

nb_iter = 10

y_onehot = torch.FloatTensor(batch_size,nb_digits)
for iteration in range(nb_iter):
    for i,(data,target) in enumerate(train_loader):
        if(i < len(train_loader) -1 ) :
            y_onehot = torch.FloatTensor(batch_size,nb_digits)
            y_onehot.zero_()
            y_onehot.scatter_(1, target.view(-1,1), 1)
            y_onehot = Variable(y_onehot,requires_grad=False) 
            data = Variable(data.view(-1 , taille_image))
            y_pred = (modele(data))
            erreur  = l(y_pred , y_onehot)
            modele.zero_grad()
            erreur.backward()
            for param in modele.parameters():
                param.data -= step * param.grad.data
            if iteration % 2 == 0 and i % 200 == 0:
                tab_err.append(erreur.data[0])
                print ("Iteration : ",iteration," MiniBatch : ",i," Erreur en train : ",erreur.data[0])

('Iteration : ', 0, ' MiniBatch : ', 0, ' Erreur en train : ', 63.10878372192383)
('Iteration : ', 0, ' MiniBatch : ', 200, ' Erreur en train : ', 18.55070686340332)
('Iteration : ', 0, ' MiniBatch : ', 400, ' Erreur en train : ', 11.562779426574707)
('Iteration : ', 0, ' MiniBatch : ', 600, ' Erreur en train : ', 11.95739459991455)
('Iteration : ', 0, ' MiniBatch : ', 800, ' Erreur en train : ', 6.686069011688232)
('Iteration : ', 2, ' MiniBatch : ', 0, ' Erreur en train : ', 4.616051197052002)
('Iteration : ', 2, ' MiniBatch : ', 200, ' Erreur en train : ', 4.49773645401001)
('Iteration : ', 2, ' MiniBatch : ', 400, ' Erreur en train : ', 6.485945224761963)
('Iteration : ', 2, ' MiniBatch : ', 600, ' Erreur en train : ', 3.323836326599121)
('Iteration : ', 2, ' MiniBatch : ', 800, ' Erreur en train : ', 4.332510471343994)
('Iteration : ', 4, ' MiniBatch : ', 0, ' Erreur en train : ', 3.3705668449401855)
('Iteration : ', 4, ' MiniBatch : ', 200, ' Erreur en train : ', 2.99356818199157

### Réseau de neuronne Classique

In [131]:
taille_image  = 28*28
profondeur = 5
nb_digits = 10
output = 200
modele1 = torch.nn.Sequential(Linear_Model_tanh(taille_image,output))  
for i in range (profondeur) :
    modele1.add_module(''+str(i+1),Linear_Model_tanh(output,output))
modele1.add_module(''+str(i+2),Linear_Model_tanh(output,nb_digits))

nb_iter = 10
l = torch.nn.MSELoss()
step = 0.005
tab_err_clas = []
tab_acc_clas = []
tab_acc_test_clas = []

y_onehot = torch.FloatTensor(batch_size,nb_digits)
for iteration in range(nb_iter):
    for i,(data,target) in enumerate(train_loader):
        if(i < len(train_loader) -1 ) :
            y_onehot = torch.FloatTensor(batch_size,nb_digits)
            y_onehot.zero_()
            y_onehot.scatter_(1, target.view(-1,1), 1)
            y_onehot = Variable(y_onehot,requires_grad=False) 
            data = Variable(data.view(-1 , taille_image))
            y_pred = (modele1(data))
            erreur  = l(y_pred , y_onehot)
            modele1.zero_grad()
            erreur.backward()
            for param in modele1.parameters():
                param.data -= step * param.grad.data
            if iteration % 2 == 0 and i % 200 == 0:
                tab_err_clas.append(erreur.data[0])
                print ("Iteration : ",iteration," MiniBatch : ",i," Erreur en train : ",erreur.data[0])

('Iteration : ', 0, ' MiniBatch : ', 0, ' Erreur en train : ', 0.10723201185464859)
('Iteration : ', 0, ' MiniBatch : ', 200, ' Erreur en train : ', 0.09255336225032806)
('Iteration : ', 0, ' MiniBatch : ', 400, ' Erreur en train : ', 0.09122177958488464)
('Iteration : ', 0, ' MiniBatch : ', 600, ' Erreur en train : ', 0.08888693153858185)
('Iteration : ', 0, ' MiniBatch : ', 800, ' Erreur en train : ', 0.08755185455083847)
('Iteration : ', 2, ' MiniBatch : ', 0, ' Erreur en train : ', 0.08337413519620895)
('Iteration : ', 2, ' MiniBatch : ', 200, ' Erreur en train : ', 0.08233673125505447)
('Iteration : ', 2, ' MiniBatch : ', 400, ' Erreur en train : ', 0.07941962778568268)
('Iteration : ', 2, ' MiniBatch : ', 600, ' Erreur en train : ', 0.07938293367624283)
('Iteration : ', 2, ' MiniBatch : ', 800, ' Erreur en train : ', 0.07825444638729095)
('Iteration : ', 4, ' MiniBatch : ', 0, ' Erreur en train : ', 0.07318016141653061)
('Iteration : ', 4, ' MiniBatch : ', 200, ' Erreur en train 

### Visualisation des courbes

In [132]:
plt.figure()
plt.title(" Courbe de l'erreur en apprentissage selon le nombre d'iterations \n profondeur = 5 ")
plt.xlabel("Nombre d'iterations batch")
plt.ylabel("Loss")
ylim(-10,70)
plt.plot(tab_err_clas,color="red", label = "Classic")
plt.plot(tab_err,color = "blue",  label = "HighWay")
plt.legend(loc = 1)
plt.show()

## Profondeur = 12

### HighWay NetWork

In [127]:
taille_image  = 28*28
nb_digits = 10
dim_highWay_network = 200
modele = torch.nn.Sequential(Linear_Model_tanh(taille_image,dim_highWay_network),HightWay_network(dim_highWay_network,dim_highWay_network,12),
                            Linear_Model_tanh(dim_highWay_network ,nb_digits))  
l = torch.nn.MSELoss(size_average=False)
step = 0.005
tab_err = []
tab_acc = []
tab_acc_test = []

nb_iter = 10

y_onehot = torch.FloatTensor(batch_size,nb_digits)
for iteration in range(nb_iter):
    for i,(data,target) in enumerate(train_loader):
        if(i < len(train_loader) -1 ) :
            y_onehot = torch.FloatTensor(batch_size,nb_digits)
            y_onehot.zero_()
            y_onehot.scatter_(1, target.view(-1,1), 1)
            y_onehot = Variable(y_onehot,requires_grad=False) 
            data = Variable(data.view(-1 , taille_image))
            y_pred = (modele(data))
            erreur  = l(y_pred , y_onehot)
            modele.zero_grad()
            erreur.backward()
            for param in modele.parameters():
                param.data -= step * param.grad.data
            if iteration % 2 == 0 and i % 200 == 0:
                tab_err.append(erreur.data[0])
                print ("Iteration : ",iteration," MiniBatch : ",i," Erreur en train : ",erreur.data[0])

('Iteration : ', 0, ' MiniBatch : ', 0, ' Erreur en train : ', 65.0546875)
('Iteration : ', 0, ' MiniBatch : ', 200, ' Erreur en train : ', 57.090030670166016)
('Iteration : ', 0, ' MiniBatch : ', 400, ' Erreur en train : ', 45.8299674987793)
('Iteration : ', 0, ' MiniBatch : ', 600, ' Erreur en train : ', 38.51225662231445)
('Iteration : ', 0, ' MiniBatch : ', 800, ' Erreur en train : ', 31.874818801879883)
('Iteration : ', 2, ' MiniBatch : ', 0, ' Erreur en train : ', 22.686025619506836)
('Iteration : ', 2, ' MiniBatch : ', 200, ' Erreur en train : ', 15.226015090942383)
('Iteration : ', 2, ' MiniBatch : ', 400, ' Erreur en train : ', 6.932811260223389)
('Iteration : ', 2, ' MiniBatch : ', 600, ' Erreur en train : ', 7.404471397399902)
('Iteration : ', 2, ' MiniBatch : ', 800, ' Erreur en train : ', 5.195773124694824)
('Iteration : ', 4, ' MiniBatch : ', 0, ' Erreur en train : ', 2.587137460708618)
('Iteration : ', 4, ' MiniBatch : ', 200, ' Erreur en train : ', 4.472926616668701)
('

### Réseau de neuronne Classique

In [126]:
taille_image  = 28*28
profondeur = 12
nb_digits = 10
output = 200
modele1 = torch.nn.Sequential(Linear_Model_tanh(taille_image,output))  
for i in range (profondeur) :
    modele1.add_module(''+str(i+1),Linear_Model_tanh(output,output))
modele1.add_module(''+str(i+2),Linear_Model_tanh(output,nb_digits))

nb_iter = 10
l = torch.nn.MSELoss(size_average=False)
step = 0.0005
tab_err_clas = []
tab_acc_clas = []
tab_acc_test_clas = []

y_onehot = torch.FloatTensor(batch_size,nb_digits)
for iteration in range(nb_iter):
    for i,(data,target) in enumerate(train_loader):
        if(i < len(train_loader) -1 ) :
            y_onehot = torch.FloatTensor(batch_size,nb_digits)
            y_onehot.zero_()
            y_onehot.scatter_(1, target.view(-1,1), 1)
            y_onehot = Variable(y_onehot,requires_grad=False) 
            data = Variable(data.view(-1 , taille_image))
            y_pred = (modele1(data))
            erreur  = l(y_pred , y_onehot)
            modele1.zero_grad()
            erreur.backward()
            for param in modele1.parameters():
                param.data -= step * param.grad.data
            if iteration % 2 == 0 and i % 200 == 0:
                tab_err_clas.append(erreur.data[0])
                print ("Iteration : ",iteration," MiniBatch : ",i," Erreur en train : ",erreur.data[0])

('Iteration : ', 0, ' MiniBatch : ', 0, ' Erreur en train : ', 65.90435791015625)
('Iteration : ', 0, ' MiniBatch : ', 200, ' Erreur en train : ', 57.62418746948242)
('Iteration : ', 0, ' MiniBatch : ', 400, ' Erreur en train : ', 57.81982421875)
('Iteration : ', 0, ' MiniBatch : ', 600, ' Erreur en train : ', 57.58229446411133)
('Iteration : ', 0, ' MiniBatch : ', 800, ' Erreur en train : ', 57.65434265136719)
('Iteration : ', 2, ' MiniBatch : ', 0, ' Erreur en train : ', 47.25616455078125)
('Iteration : ', 2, ' MiniBatch : ', 200, ' Erreur en train : ', 43.54288864135742)
('Iteration : ', 2, ' MiniBatch : ', 400, ' Erreur en train : ', 36.70328140258789)
('Iteration : ', 2, ' MiniBatch : ', 600, ' Erreur en train : ', 37.7913703918457)
('Iteration : ', 2, ' MiniBatch : ', 800, ' Erreur en train : ', 34.653526306152344)
('Iteration : ', 4, ' MiniBatch : ', 0, ' Erreur en train : ', 33.97379684448242)
('Iteration : ', 4, ' MiniBatch : ', 200, ' Erreur en train : ', 31.75240707397461)
(

### Visualisation des courbes

In [129]:
plt.figure()
plt.title(" Courbe de l'erreur en apprentissage selon le nombre d'iterations \n profondeur = 12 ")
plt.xlabel("Nombre d'iterations batch")
plt.ylabel("Loss")
plt.plot(tab_err_clas,color="red", label = "Classic")
plt.plot(tab_err,color = "blue",  label = "HighWay")
plt.legend(loc = 1)
plt.show()